In [ ]:
from datetime import datetime
import json
import pandas as pd
import numpy as np
from f_display_and_feedback import *
from f_pichunter_star import *
from f_polyquery_msed_logscale import *
from f_polyadic_sed import *
from f_process_data import *
from f_rocchio import *
from f_svm import *
from f_process_data import *
import h5py
import tqdm 
import random
import pickle


In [ ]:

hdf5_file_path= '/home/francescascotti/dev/interfaccia/indexed_file.h5'
hdf5_file = h5py.File(hdf5_file_path, 'r')
features = hdf5_file['indexed_data/data'][:]
ids = hdf5_file['indexed_data/ids'][:]
hdf5_file.close()
      
# pickle load  /home/francescascotti/dev/interfaccia/indexed_data_logistic 
     
with open('/home/francescascotti/dev/interfaccia/indexed_data_logistic', 'rb') as file:
    features_logistic = pickle.load(file)
  
    
shot_labels_query, queries =  process_data( '/home/francescascotti/data/avsGT/avs_gt_visione_mapping.csv','/home/francescascotti/data/avsGT/avs_gt_visione_mapping_query_judgment.csv',threshold_1=200, threshold_0=1400)
features = pd.DataFrame(features).transpose()
features.columns = [id.decode() if isinstance(id, bytes) else id for id in ids]
features_logistic = pd.DataFrame(features_logistic).transpose()
features_logistic.columns = features.columns

/home/francescascotti/dev/interfaccia/f_process_data.py:11: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_visione_mapping = pd.read_csv(csv_mapping_path, dtype={'column_name': str})


In [ ]:

lista_10_labeled_positive = []  # Stores 10 positive samples per query
lista_10_labeled_negative = []  # Stores 10 negative samples per query
lista_total = []  # Stores both positive and negative samples per query

for query in queries:
    seed = 42 
    
    # Determine the sample size (max 10,000 or the number of available samples)
    sample_size = min(10000, len([el[0] for el in shot_labels_query.keys() if el[1] == query]))
    
    # Extract feature IDs that belong to the current query
    features_id_accepted = random.sample(
        [el[0] for el in shot_labels_query.keys() if el[1] == query], sample_size
    )
    
    # Select 5 positive and 5 negative samples randomly from the accepted features
    positive_first_display = random.sample(
        [i for i in features_id_accepted if shot_labels_query[(i, query)] == 1], 5
    ) 
    negative_first_display = random.sample(
        [i for i in features_id_accepted if shot_labels_query[(i, query)] == 0], 5
    ) 
    
    # Append the selected samples to the corresponding lists
    lista_10_labeled_positive.append(positive_first_display)
    lista_10_labeled_negative.append(negative_first_display)
    lista_total.append(positive_first_display + negative_first_display)

# Save positive and negative labeled lists as JSON files
with open('/home/francescascotti/dev/interfaccia/lista_10_labeled_positive.json', 'w') as f:
    json.dump(lista_10_labeled_positive, f)

with open('/home/francescascotti/dev/interfaccia/lista_10_labeled_negative.json', 'w') as f:
    json.dump(lista_10_labeled_negative, f)

# Count the total number of occurrences and track unique feature IDs
count = 0
duplicates = []  # Stores unique feature IDs
ids_to_keep = []  # Stores all selected feature IDs

for el in lista_total:
    for aa in el:
        ids_to_keep.append(aa)  # Append feature ID to tracking list
        if aa not in duplicates:
            duplicates.append(aa)  # Add to unique list if not already present
        count += 1  # Increment total count

# Extract dataset using selected feature IDs
dataset_800 = features[ids_to_keep]

target_columns = 10000  # Target number of columns in the final dataset
current_columns = len(dataset_800.columns)  # Current number of columns
columns_to_add = target_columns - current_columns  # Calculate missing columns

# If additional columns are needed, randomly select them from available features
if columns_to_add > 0:
    new_features_ids = random.sample(
        [el for el in features.columns if el not in dataset_800.columns], columns_to_add
    )
    new_features = features[new_features_ids]  # Extract the additional features
    
    # Concatenate the new features to the dataset
    dataset_800 = pd.concat([dataset_800, new_features], axis=1)

# Save the final dataset as a CSV file
dataset_800.to_csv('/home/francescascotti/dev/interfaccia/data_10000_df.csv', index=False)


In [ ]:


# Remove duplicate queries by converting the list to a set and back to a list
queries = list(set(queries))

# Load dataset containing 10,000 data points
data_10000_df = pd.read_csv('/home/francescascotti/dev/interfaccia/data_10000_df.csv')

# Select only relevant columns for logistic regression features
data_10000_df_logistic = features_logistic[data_10000_df.columns]

# Load labeled positive and negative examples from JSON files
with open('/home/francescascotti/dev/interfaccia/lista_10_labeled_positive.json', 'r') as f:
    lista_10_labeled_positive = json.load(f)
with open('/home/francescascotti/dev/interfaccia/lista_10_labeled_negative.json', 'r') as f:
    lista_10_labeled_negative = json.load(f)

# Initialize lists to store execution times of different methods
rocchio_time, svm_time, pichunter_time, pichunter_star_time, polyadic_sed_time, polyadic_msed_time = [], [], [], [], [], []

# Iterate over each query in the list
for i in range(len(queries)):
    start_time = datetime.now()  # Start timing data preparation
    query = queries[i]  # Current query
    seed = 42  # Set random seed for reproducibility
    print("--------------------")
    print("Query: ", query)
    
    random.seed(seed)
    
    # Retrieve positive and negative samples for initial display
    positive_first_display = lista_10_labeled_positive[i]
    negative_first_display = lista_10_labeled_negative[i]
    
    # Create the initial display dataset with labeled examples
    display_df_1 = pd.concat([features[positive_first_display], features[negative_first_display]], axis=1)
    
    # Randomly select 90 additional features for display
    new_features_id = random.sample([el for el in features.columns], 90)
    display_df_finale = pd.concat([display_df_1, features[new_features_id]], axis=1)
    display_df_final_logistic = features_logistic[display_df_finale.columns]
    
    # Ensure the dataset contains the expected number of features
    if len(list(set(display_df_finale.columns))) != 100:
        print('Error: Incorrect number of features in display dataset')
    if len(list(set(data_10000_df.columns))) != 10000:
        print('Error: Incorrect number of features in full dataset')
    
    end_time = datetime.now()  # End timing data preparation
    print("Data preparation time: ", end_time - start_time)
    print("--------------------")
    
    ### Run different relevance feedback methods and store execution times ###
    
    # ROCCHIO Method
    _, _, _, time_of_search_rocchio, _, _ = rocchio_single_step(
        data_10000_df, display_df_finale, positive_first_display,
        negative_first_display, alpha=0.75, beta=1, gamma=0.75,
        fun_name="euclidean", initial_query=None
    )
    rocchio_time.append(time_of_search_rocchio)
    
    # SVM Method
    _, _, time_of_search_svm, _, _, _ = svm_single_step(
        data_10000_df, display_df_finale, positive_first_display,
        negative_first_display, initial_scores=None, alpha=0.75, beta=0.25
    )
    svm_time.append(time_of_search_svm)
    
    # PICHUNTER_STAR Method
    _, _, time_of_search_pic_star, _, _ = pichunter_single_step(
        data_10000_df, display_df_finale, positive_first_display, negative_first_display,
        fun_name="softmin", initial_prob=0, temperature=82.10553
    )
    pichunter_star_time.append(time_of_search_pic_star)
    
    # PICHUNTER Method
    _, _, time_of_search_pic, _, _ = pichunter_single_step(
        data_10000_df, display_df_finale, positive_first_display, [],
        fun_name="softmin", initial_prob=0, temperature=82.10553
    )
    pichunter_time.append(time_of_search_pic)
    
    # POLYADIC SED Method
    _, _, _, _, time_of_search_sed, _, _ = poly_sed_logscale_single_step(
        data_10000_df_logistic, display_df_final_logistic,
        positive_first_display, negative_first_display,
        precomputed_dict_initial=None, alpha=0.75, beta=1, gamma=0.75,
        initial_query=None, initial_scores=None, entropy_dict=None
    )
    polyadic_sed_time.append(time_of_search_sed)
    
    # POLYADIC MSED Method
    _, _, _, _, time_of_search_msed, _, _ = poly_msed_logscale_single_step(
        data_10000_df_logistic, display_df_final_logistic,
        positive_first_display, negative_first_display,
        precomputed_dict_initial=None, alpha=0.75, beta=1, gamma=0.75,
        initial_query=None, initial_scores=None, entropy_dict=None
    )
    polyadic_msed_time.append(time_of_search_msed)

# Uncomment the following lines if you need to save the execution times to JSON files
# with open('/home/francescascotti/dev/interfaccia/time_dict_pichunter.json', 'w') as file:
#     json.dump(time_dict_pichunter, file, indent=4)
# with open('/home/francescascotti/dev/interfaccia/time_dict_svm.json', 'w') as file:
#     json.dump(time_dict_svm, file, indent=4)
# with open('/home/francescascotti/dev/interfaccia/time_dict_rocchio.json', 'w') as file:
#     json.dump(time_dict_rocchio, file, indent=4)
# with open('/home/francescascotti/dev/interfaccia/time_dict_polyadic2.json', 'w') as file:
#     json.dump(time_dict_sed_logscale, file, indent=4)
# with open('/home/francescascotti/dev/interfaccia/time_dict_msed.json', 'w') as file:
#     json.dump(time_dict_msed_logscale, file, indent=4)


--------------------
Query:  1591
Data preparation time:  0:00:00.122084
--------------------
--------------------
Query:  1592
Data preparation time:  0:00:00.123514
--------------------
--------------------
Query:  1593
Data preparation time:  0:00:00.121902
--------------------
--------------------
Query:  1594
Data preparation time:  0:00:00.122178
--------------------
--------------------
Query:  1595
Data preparation time:  0:00:00.121326
--------------------
--------------------
Query:  1597
Data preparation time:  0:00:00.121951
--------------------
--------------------
Query:  1598
Data preparation time:  0:00:00.122112
--------------------
--------------------
Query:  1599
Data preparation time:  0:00:00.121726
--------------------
--------------------
Query:  1600
Data preparation time:  0:00:00.122498
--------------------
--------------------
Query:  1601
Data preparation time:  0:00:00.127621
--------------------
--------------------
Query:  1602
Data preparation time:  0:

In [9]:
print('mean rocchio time:', np.mean(rocchio_time).total_seconds())
print('mean svm time:', np.mean(svm_time).total_seconds())
print('mean pichunter time:', np.mean(pichunter_time).total_seconds())
print('mean pichunter star time:', np.mean(pichunter_star_time).total_seconds())
print('mean polyadic sed time:', np.mean(polyadic_sed_time).total_seconds())
print('mean polyadic msed time:', np.mean(polyadic_msed_time).total_seconds())

mean rocchio time: 0.052747
mean svm time: 0.045806
mean pichunter time: 0.09063
mean pichunter star time: 0.095069
mean polyadic sed time: 2.819505
mean polyadic msed time: 2.763181
